In [1]:
##########################################################################################
#                                                                                        #
#  888888b.  8888888 .d88888b.   .d8888b.     .d8888b.   .d8888b.      d8888      d8888  #
#  888  "88b   888  d88P" "Y88b d88P  Y88b   d88P  Y88b d88P  Y88b    d8P888     d8P888  #
#  888  .88P   888  888     888 Y88b.        888        888          d8P 888    d8P 888  #
#  8888888K.   888  888     888  "Y888b.     888d888b.  888d888b.   d8P  888   d8P  888  #
#  888  "Y88b  888  888     888     "Y88b.   888P "Y88b 888P "Y88b d88   888  d88   888  #
#  888    888  888  888     888       "888   888    888 888    888 8888888888 8888888888 #
#  888   d88P  888  Y88b. .d88P Y88b  d88P   Y88b  d88P Y88b  d88P       888        888  #
#  8888888P" 8888888 "Y88888P"   "Y8888P"     "Y8888P"   "Y8888P"        888        888  #
#                                                                                        # 
##########################################################################################
#
# Wrangle Synthetic Medical Record Data -- Part 2
#
##########################################################################################

## Fetch data
(This is the same data used in the previous homework--no need to re-fetch it)
Go here: https://drive.google.com/drive/folders/0B5Wu8z15TIbiLUhvcWxaMDJCRGs

Get this data set: 10000-Patients.zip

unzip it into the Data folder

In [1]:
import pandas as pd
import pylab as plt
import numpy as np
import datetime as dt

In [2]:
##  ###################################################################################################################
#
# You do:  
#    Ensure you have SQLite3 available to your system.  If not, install it via pip, conda, etc.
import sqlite3

In [3]:
# Read data in to data frames

p = pd.read_csv('../BIOS6644/Data/PatientCorePopulatedTable.txt', header=0,sep = '\t')
a = pd.read_csv('../BIOS6644/Data/AdmissionsCorePopulatedTable.txt', header=0, sep='\t')
d = pd.read_csv('../BIOS6644/Data/AdmissionsDiagnosesCorePopulatedTable.txt', header=0, sep='\t')  
l = pd.read_csv('../BIOS6644/Data/LabsCorePopulatedTable.txt', header=0, sep='\t')            


In [5]:
#p.loc[p.PatientID=='915BC24E-8C44-4D33-A386-CEA965B83F32']

In [4]:
# Convert date columns to pandas date objects
# Show the type
type(p.PatientDateOfBirth[3])

for table in [p,a,d,l]:
    for col in table.columns:
        if 'date' in col.lower():
            print('Converting {}'.format(col))
            table[col] = pd.to_datetime(table[col])

Converting PatientDateOfBirth
Converting AdmissionStartDate
Converting AdmissionEndDate
Converting LabDateTime


 SQLite3 is a "lightweight" database management system.  Within Python you create a "connection" to the dabaase 
 (here we called it "conn") and you can then send SQL queries to the connection, getting standard data frames in 
 return.



In [5]:

conn = sqlite3.connect(':memory:')  # this makes an "in memory" database that isn't written to a disk.  You 
                                    # can replace this with a file name if you want to store the data for later
                                    # use.

print('Loading data into in-memory db--this could take a minute or two')
l.to_sql('L', conn, if_exists='replace')
d.to_sql('D', conn, if_exists='replace')
a.to_sql('A', conn, if_exists='replace')
p.to_sql('P', conn, if_exists='replace')

Loading data into in-memory db--this could take a minute or two


## Basic SQL queries


In [8]:
# To query via SQLite3, write a query as a Python string and "execute it" read_sql_query() method.
#
# Note:  If you are accessing data already in a different database managemnt system (MySQL/Access/etc) there are
#  interfaces available which give you a "conn" to that data as well.  You're not limited to SQLite with this
#  method.


sql = """
SELECT * from L
LIMIT 10
"""

df2 = pd.read_sql_query(sql, conn)
df2



,index,PatientID,AdmissionID,LabName,LabValue,LabUnits,LabDateTime
0,0,915BC24E-8C44-4D33-A386-CEA965B83F32,1,CBC: HEMATOCRIT,40.7,%,1946-09-07 22:20:26.677000
1,1,915BC24E-8C44-4D33-A386-CEA965B83F32,1,METABOLIC: ANION GAP,8.4,mmol/L,1946-09-07 11:52:58.600000
2,2,915BC24E-8C44-4D33-A386-CEA965B83F32,1,CBC: LYMPHOCYTES,4.7,k/cumm,1946-09-07 06:08:57.303000
3,3,915BC24E-8C44-4D33-A386-CEA965B83F32,1,CBC: HEMOGLOBIN,15.9,gm/dl,1946-09-07 19:16:10.057000
4,4,915BC24E-8C44-4D33-A386-CEA965B83F32,1,METABOLIC: SODIUM,146.6,mmol/L,1946-09-07 14:03:11.003000
5,5,915BC24E-8C44-4D33-A386-CEA965B83F32,1,METABOLIC: ALBUMIN,3.3,gm/dL,1946-09-08 04:06:53.967000
6,6,915BC24E-8C44-4D33-A386-CEA965B83F32,1,METABOLIC: BUN,17.1,mg/dL,1946-09-07 19:41:39.247000
7,7,915BC24E-8C44-4D33-A386-CEA965B83F32,1,CBC: NEUTROPHILS,8.4,k/cumm,1946-09-07 05:33:34.320000
8,8,915BC24E-8C44-4D33-A386-CEA965B83F32,1,METABOLIC: CALCIUM,8.7,mg/dL,1946-09-07 16:23:41.213000
9,9,915BC24E-8C44-4D33-A386-CEA965B83F32,1,METABOLIC: GLUCOSE,110.5,mg/dL,1946-09-07 09:40:39.467000


In [6]:

# Use the WHERE statement

# Example: Query all labs from admission-id #3

sql = """
SELECT *
FROM L
WHERE AdmissionID==3;
"""
pd.read_sql_query(sql, conn)
#df2


,index,PatientID,AdmissionID,LabName,LabValue,LabUnits,LabDateTime
0,163,915BC24E-8C44-4D33-A386-CEA965B83F32,3,CBC: MCHC,31.6,g/dl,2008-10-08 10:11:47.543000
1,164,915BC24E-8C44-4D33-A386-CEA965B83F32,3,CBC: MCH,29.6,pg,2008-10-08 17:55:31.247000
2,165,915BC24E-8C44-4D33-A386-CEA965B83F32,3,METABOLIC: CREATININE,0.6,mg/dL,2008-10-08 15:36:59.533000
3,166,915BC24E-8C44-4D33-A386-CEA965B83F32,3,CBC: WHITE BLOOD CELL COUNT,11.4,k/cumm,2008-10-08 09:31:41.360000
4,167,915BC24E-8C44-4D33-A386-CEA965B83F32,3,METABOLIC: BUN,14.5,mg/dL,2008-10-08 07:56:07.697000
5,168,915BC24E-8C44-4D33-A386-CEA965B83F32,3,CBC: RDW,11.1,%,2008-10-08 03:24:55.450000
6,169,915BC24E-8C44-4D33-A386-CEA965B83F32,3,URINALYSIS: RED BLOOD CELLS,0.3,rbc/hpf,2008-10-08 23:39:43.667000
7,170,915BC24E-8C44-4D33-A386-CEA965B83F32,3,CBC: ABSOLUTE LYMPHOCYTES,23.1,%,2008-10-08 18:41:18.467000
8,171,915BC24E-8C44-4D33-A386-CEA965B83F32,3,METABOLIC: CARBON DIOXIDE,20.9,mmol/L,2008-10-08 04:50:40.677000
9,172,915BC24E-8C44-4D33-A386-CEA965B83F32,3,METABOLIC: GLUCOSE,93.1,mg/dL,2008-10-08 16:40:24.897000


In [29]:
# Example: 
# Query just the time and value for CBC: MCHC labs from admission-id #3, ensuring they are sorted in time order


sql = """
SELECT LabDateTime AS Time, LabValue as Value
FROM L
WHERE AdmissionID=3 AND LabName LIKE "%MCHC"
ORDER BY Time
"""

df3 = pd.read_sql_query(sql, conn)
df3.head()

,Time,Value
0,1948-10-17 08:30:55.433000,35.8
1,1948-10-18 23:34:59.213000,39.9
2,1948-10-19 09:20:36.913000,31.1
3,1948-10-21 22:42:59.537000,33.0
4,1948-10-23 05:08:27.097000,31.4


In [11]:
# You would like to explore the relationship between lab-orders and depression
#
###################################################################################################################
#
# You do:
#  1) 
#   Generate a list of all (unique) diagnosis codes with the term "Major depressive disorder" in the description
#
#  2) Using a JOIN statement, join the Diagosis and Labs tables on PatientID and print the first 40 results

In [1]:
# 1)


In [2]:
# 2) 


In [3]:

###################################################################################################################
#
# You do:
#  1) Create a list of each lab name ordered and it's count for any patient who also had a diagnosis which 
#     includes the term "Major depressive disorder"


